In [3]:
import statsmodels as sm
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.api import VAR

/anaconda/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [65]:
x = pd.read_csv('/Users/andrewslottje/Documents/x.csv', parse_dates=True)
x = x.rename(columns={"[TimeStamp]":"Timestamp"})
y = pd.read_csv('/Users/andrewslottje/Documents/y.csv', parse_dates=True)

In [66]:
x.index = pd.DatetimeIndex(x.Timestamp)
#x['ETHreturnindicator']=np.sign(x['ETHreturn'])
x['ETHspread_d']=np.log(x['ETHspread']/x['ETHspread'].shift(1))
x['ETHvolume_d']=np.log(x['ETHvolume']/x['ETHvolume'].shift(1))
x.loc[x.ETHvolume_d.isnull(),'ETHvolume_d']=0.0
x.loc[x.ETHspread_d.isnull(),'ETHspread_d']=0.0
x['ETHreturnindicator']=np.sign(x['ETHreturn'])
x=x[~x.isnull()]
data = x[['ETHreturn','ETHspread_d','ETHvolume_d','XRPreturn','LTCreturn','DASHreturn','XMRreturn','ETHreturnindicator']]

/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:3: RuntimeWarning: invalid value encountered in log
  app.launch_new_instance()
/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:4: RuntimeWarning: invalid value encountered in log


In [67]:
model=VAR(data)
#model.select_order(50)
results = model.fit(3)
data['ETHpredict']=results.fittedvalues['ETHreturn']
data['ETHreturn_resid']=(data['ETHreturn']-results.fittedvalues['ETHreturn'])
data['ETHmatch']=np.sign(data['ETHreturn']*results.fittedvalues['ETHreturn'])
data['ETHreturn_resid_chg']=data['ETHreturn_resid'].diff(1)

/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-cop

In [64]:
np.sum(data['ETHmatch']==1)/10938

0.51517644907661364

In [68]:
data.to_csv('/Users/andrewslottje/Documents/train_x.csv')